In [11]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import os 
import sys 

# TODO: remove '..' if run from root folder
home = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
pths = [os.path.join(home, 'astrobot', 'astrobot'), os.path.join(home, 'astrobot', 'tests')]

for pth in pths: 
    if pth not in sys.path: sys.path.insert(0, pth)

from astrobot import pmf

import numpy as np 
import pandas as pd
from pprint import pprint 
import matplotlib.pyplot as plt 

def pp(x, *args): 
    return pprint(x) if not args else print(x, *args)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
DATADIR = "C:/users/admin/master/astrobot/resources/rescomstock/comstock"
assert os.path.isdir(DATADIR)
metacom_fpath = os.path.join(DATADIR, "metadata_comstock.tsv")
_comdf = pd.read_csv(metacom_fpath, sep='\t')
_comdf = _comdf.dropna(axis=0, how="any")
_comdf.head(2)

,bldg_id,applicability,in.upgrade_name,in.aspect_ratio,in.county,in.building_type,in.rotation,in.number_of_stories,in.sqft,in.hvac_system_type,...,out.electricity.total.energy_consumption_intensity,out.site_energy.total.energy_consumption,out.site_energy.total.energy_consumption_intensity,out.natural_gas.total.energy_consumption,out.natural_gas.total.energy_consumption_intensity,out.other_fuel.total.energy_consumption,out.other_fuel.total.energy_consumption_intensity,upgrade,weight,metadata_index
0,105,True,Baseline,2.0,G0100890,SmallOffice,270.0,1.0,17500.0,PSZ-AC with electric coil,...,8.988889,536.748835,0.030671,0.0,0.0,0.0,0.0,0,7.041741,0
1,170,True,Baseline,3.0,G0100690,SmallOffice,270.0,1.0,3000.0,PSZ-AC with electric coil,...,10.487037,107.349767,0.035783,0.0,0.0,0.0,0.0,0,7.041741,1


In [6]:
var = "weight"
[col for col in _comdf.columns if var in col]

['weight']

In [7]:
comdf = _comdf.copy()
# pp([c for c in _comdf.columns if "in." in c])

y_lbls = ["eui"] 
X_lbls = ["cz_cat", "cz", "btype_cat", "btype", "weight", "metadata_index"] 
comdf[["cz_cat", "btype_cat"]] = comdf[["in.climate_zone_ashrae_2004", "in.building_type"]]
_mbtu_arr, _sqft_arr = \
    comdf["out.site_energy.total.energy_consumption"], comdf["in.sqft"]
comdf[y_lbls[0]] = (_mbtu_arr * 1e6 / 3412) / (_sqft_arr / 10.764)  # Convert MBTU, ft2 -> kWh/m2

# Integer encoding for cz categories
cz_dict = {cz:i for i, cz in enumerate(np.unique(comdf["cz_cat"]))}
comdf["cz"] = [cz_dict[cz] for cz in comdf["cz_cat"]]
 
# Integer encoding for btype
btype_dict = {bt:i for i, bt in enumerate(np.unique(comdf["btype_cat"]))}
comdf["btype"] = [btype_dict[bt] for bt in comdf["btype_cat"]]

comdf = comdf[X_lbls + y_lbls]
comdf.head(3)

,cz_cat,cz,btype_cat,btype,weight,metadata_index,eui
0,3A,3,SmallOffice,12,7.041741,0,96.760416
1,3A,3,SmallOffice,12,7.041741,1,112.887152
2,3A,3,RetailStripmall,9,1.947971,2,455.818507


In [14]:
from tests.pmf_test import test_make_pmf1, test_make_bin_arr

from scipy.stats._binned_statistic import binned_statistic
pd.set_option("display.precision", 6)

# data = pd.DataFrame(
#         {"qs": np.array([.1, .2, .2, .3, .1]),
#          "ps": np.array([2., 1., 1., 2., 3.])})
# q = pmf.Pmf1(data.qs, data.ps, bins=3)

# Joint Pmf
btype = "SmallOffice"
cz = "3A"
data = comdf.query("btype_cat == '{}' & cz_cat == '{}'".format(btype, cz))

# eui = pmf.Pmf1(data.eui, data.weight, bins=25)
# pmf.plt_hist(eui, width=15); 


test_make_pmf1() 
test_make_bin_arr()




